In [1]:
import cobra
import os
from cobra import Model, Reaction, Metabolite
import pandas as pd

path = "/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm"
data_path = "/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/data"
os.chdir(data_path+'/other_models')

model = cobra.io.read_sbml_model("iPfal18_pregapfilling.xml")
os.chdir(data_path)
universal = cobra.io.load_json_model('universal_model_may2018.json')

In [2]:
# cobra.manipulation.modify.convert_to_irreversible(model)
# for rxn in universal.reactions:
#     if rxn.id.startswith('EX_'):
#         print(rxn)
#         universal.remove_reactions([rxn])

In [3]:
# cobra.manipulation.modify.convert_to_irreversible(model)
# print ('Note: Model is converted to irreversible. See cobra.manipulation.modify.convert_to_irreversible(cobra_model)')
model.solver = 'gurobi'
model.slim_optimize()
og_pf = model.copy()


Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Parameter Method unchanged
   Value: 0  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1


In [4]:
# map BiGG to ChEBI for translating data to model nomenclature
conversion = pd.read_table("http://bigg.ucsd.edu/static/namespace/bigg_models_metabolites.txt", sep = "\t")
df = conversion.drop(labels=['bigg_id','old_bigg_ids','model_list'], axis = 1)
df = df.drop_duplicates(keep = 'first')
df.reset_index(inplace=True)
df = df.drop(labels=['index'], axis = 1)
new_dict = dict()
for x in range(0,len(df['database_links'])):
    y = str(df['database_links'][x])
    valu = df['universal_bigg_id'][x]
    if '.org/chebi/CHEBI:' in y:
        for w in range(0,(len(y.split('.org/chebi/CHEBI:')) -1)):
            z = y.split('.org/chebi/CHEBI:')[w+1]
            if ';' in z:
                z = z.split(";")[0]
            new_dict[z] = valu

In [5]:
# these are mets CONSUMMED by parasite, see README

def get_import_reaction_only(model,met_id_e):
    
    # identify transport reactions that move met_id into the cell 
    ###### model is an irreversible cobra model
    # met_id_e is a string containing a metabolite id with extracellular localization(i.e. 'h_e')
    
    reaction_is = []
    compartmentalized_e_met = met_id_e
    compartmentalized_c_met = met_id_e[:-2]+"_c"

    if compartmentalized_c_met in [x.id for x in model.metabolites]:
        mets_reactions_c = model.metabolites.get_by_id(compartmentalized_c_met).reactions
    else:
        mets_reactions_c = []
    if compartmentalized_e_met in [x.id for x in model.metabolites]:
        mets_reactions_e = model.metabolites.get_by_id(compartmentalized_e_met).reactions
    else:
        mets_reactions_e = []
    if bool(set(mets_reactions_c)) and bool(set(mets_reactions_e)):
        transport = mets_reactions_c.intersection(mets_reactions_e)
    else:
        transport = []
    if bool(transport):
        for x in transport:
            if compartmentalized_c_met in [y.id for y in x.products]:
                reaction_is.append(x.id)
    return(reaction_is)

mets_consumed = ['dttp', 'gtp', '5mta'] # last met is 5'-S-methyl-5'-thioadenosine
# reminder: capitalization matters

for x in mets_consumed:
    x = x +'_e'
    print(len(get_import_reaction_only(model,x)))
    if len(get_import_reaction_only(model,x)) == 0:
        print(x + ' cannot be imported yet, must add this import reaction')
    

0
dttp_e cannot be imported yet, must add this import reaction
0
gtp_e cannot be imported yet, must add this import reaction
0
5mta_e cannot be imported yet, must add this import reaction


In [6]:

os.chdir(data_path)
universal2 = cobra.io.load_json_model('universal_model_may2018.json')

# add metabolites
model.add_metabolites(universal2.metabolites.get_by_id('dttp_e'))
model.metabolites.get_by_id('dttp_e').formula = universal2.metabolites.get_by_id('dttp_e').formula
model.metabolites.get_by_id('dttp_e').name = universal2.metabolites.get_by_id('dttp_e').name
model.metabolites.get_by_id('dttp_e').compartment = 'e'

model.add_metabolites(universal2.metabolites.get_by_id('gtp_e'))
model.metabolites.get_by_id('gtp_e').formula = universal2.metabolites.get_by_id('gtp_e').formula
model.metabolites.get_by_id('gtp_e').name = universal2.metabolites.get_by_id('gtp_e').name
model.metabolites.get_by_id('gtp_e').compartment = 'e'

model.add_metabolites(universal2.metabolites.get_by_id('5mta_e'))
model.metabolites.get_by_id('5mta_e').formula = universal2.metabolites.get_by_id('5mta_e').formula
model.metabolites.get_by_id('5mta_e').name = universal2.metabolites.get_by_id('5mta_e').name
model.metabolites.get_by_id('5mta_e').compartment = 'e'

# Add reactions
print(len(model.reactions))
# 5mta import and exchange
model.add_reactions([universal2.reactions.get_by_id('5MTAte'),universal2.reactions.get_by_id('EX_5mta_e')])
# GTP exchange
model.add_reactions([universal2.reactions.get_by_id('EX_gtp_e')])
# GTP import
my_reaction = Reaction('GTPt')
my_reaction.name = 'GTP transport'
my_reaction.subsystem = 'transport'
model.add_reactions([my_reaction])
my_reaction.reaction = 'gtp_e --> gtp_c'
# dTTP exchange and import
model.add_reactions([universal2.reactions.get_by_id('EX_dttp_e'),universal2.reactions.get_by_id('DTTPt')])

for x in mets_consumed:
    x = x +'_e'
    if len(get_import_reaction_only(model,x)) == 0:
        print(x + ' cannot be imported yet, must add this import reaction')
    

1196


In [7]:
# these are mets PRODUCED by parasite, see README
import pandas as pd
mets_down = pd.read_table("/Users/maureencarey/local_documents/work/metabolomics_integration/data/PlasmoDB_metabolomics_DOWN.txt",
             sep = "\t")
ChEBI_ids = [x.split(':')[1] for x in mets_down['[ChEBI ID]']]

# convert produced mets from ChEBI to BiGG
l = dict()
for x in ChEBI_ids:
    if str(x) in new_dict.keys():
        l[x] = new_dict[str(x)]
    else:
        l[x] = None
        
# make metabolite references compatible string searching thru the model
measured_mets = list(set([x+'_e' for x in l.values() if x])) # remove dups (due to protonated states)

In [8]:
# TAKEN FROM MEMOTE
transport_reactions = [] 
transport_reactions_UNI = [] 

from collections import defaultdict
from six import iteritems, itervalues

def find_biomass_reaction(model):
    """
    Return a list of the biomass reaction(s) of the model.
    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    """
    return [rxn for rxn in model.reactions if "biomass" in rxn.id.lower()]

def find_transported_elements(rxn):
    """
    Return a dictionary showing the amount of transported elements of a rxn.

    Collects the elements for each metabolite participating in a reaction,
    multiplies the amount by the metabolite's stoichiometry in the reaction and
    bins the result according to the compartment that metabolite is in. This
    produces a dictionary of dictionaries such as this
    ``{'p': {'C': -1, 'H': -4}, c: {'C': 1, 'H': 4}}`` which shows the
    transported entities. This dictionary is then simplified to only include
    the non-zero elements of one single compartment i.e. showing the precise
    elements that are transported.

    Parameters
    ----------
    rxn : cobra.Reaction
        Any cobra.Reaction containing metabolites.

    """
    element_dist = defaultdict()
    # Collecting elements for each metabolite.
    for met in rxn.metabolites:
        if met.compartment not in element_dist:
            # Multiplication by the metabolite stoichiometry.
            element_dist[met.compartment] = \
                {k: v * rxn.metabolites[met]
                 for (k, v) in iteritems(met.elements)}
        else:
            x = {k: v * rxn.metabolites[met] for (k, v) in
                 iteritems(met.elements)}
            y = element_dist[met.compartment]
            element_dist[met.compartment] = \
                {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}
    delta_dict = defaultdict()
    # Simplification of the resulting dictionary of dictionaries.
    for elements in itervalues(element_dist):
        delta_dict.update(elements)
    # Only non-zero values get included in the returned delta-dict.
    delta_dict = {k: abs(v) for (k, v) in iteritems(delta_dict) if v != 0}
    return delta_dict

for rxn in model.reactions: # MODEL
    # Collecting criteria to classify transporters by.
    rxn_reactants = set([met.formula for met in rxn.reactants])
    rxn_products = set([met.formula for met in rxn.products])
    # Looking for formulas that stay the same on both side of the reaction.
    transported_mets = [formula for formula in rxn_reactants if formula in rxn_products]
    # Collect information on the elemental differences between
    # compartments in the reaction.
    delta_dicts = find_transported_elements(rxn)
    non_zero_array = [v for (k, v) in iteritems(delta_dicts) if v != 0]
    # Weeding out reactions such as oxidoreductases where no net
    # transport of Hydrogen is occurring, but rather just an exchange of
    # electrons or charges effecting a change in protonation.
    if set(transported_mets) != set('H') and list(
        delta_dicts.keys()
    ) == ['H']:
        pass
    # All other reactions for which the amount of transported elements is
    # not zero, which are not part of the model's exchange nor
    # biomass reactions, are defined as transport reactions.
    # This includes reactions where the transported metabolite reacts with
    # a carrier molecule.
    
    ## updated 
    elif sum(non_zero_array) and rxn not in find_biomass_reaction(model):
        if rxn.id.startswith('EX_'):
            temp = rxn.id
        else:
            transport_reactions.append(rxn)
        
for rxn in universal.reactions: # UNIVERSAL
    # Collecting criteria to classify transporters by.
    rxn_reactants = set([met.formula for met in rxn.reactants])
    rxn_products = set([met.formula for met in rxn.products])
    # Looking for formulas that stay the same on both side of the reaction.
    transported_mets = [formula for formula in rxn_reactants if formula in rxn_products]
    # Collect information on the elemental differences between
    # compartments in the reaction.
    delta_dicts = find_transported_elements(rxn)
    non_zero_array = [v for (k, v) in iteritems(delta_dicts) if v != 0]
    # Weeding out reactions such as oxidoreductases where no net
    # transport of Hydrogen is occurring, but rather just an exchange of
    # electrons or charges effecting a change in protonation.
    if set(transported_mets) != set('H') and list(
        delta_dicts.keys()
    ) == ['H']:
        pass
    # All other reactions for which the amount of transported elements is
    # not zero, which are not part of the model's exchange nor
    # biomass reactions, are defined as transport reactions.
    # This includes reactions where the transported metabolite reacts with
    # a carrier molecule.
    elif sum(non_zero_array) and rxn not in find_biomass_reaction(universal):
        if rxn.id.startswith('EX_'):
            temp = rxn.id
        else: 
            transport_reactions_UNI.append(rxn)

In [9]:
# find transport reactions that export measured metabolites (exclude import reactions)
transport_w_measured_products = []
for rxn in transport_reactions:
    for x in rxn.products:
        if x.id in measured_mets:
            transport_w_measured_products.append(rxn)
transport_w_measured_products = list(set(transport_w_measured_products)) # remove duplicates

# find transport reactions that export measured metabolites IN UNIVERSAL REACTION
transport_w_measured_products_UNI = []
for rxn in transport_reactions_UNI:
    for x in rxn.products:
        if x.id in measured_mets:
            transport_w_measured_products_UNI.append(rxn)
            
transport_w_measured_products_UNI = list(set(transport_w_measured_products_UNI)) # remove duplicates

In [10]:
# must add transport reactions for the rest of the metabolites
l = list()
l_not = list()
good= list()
print('mets not in model')
for met in measured_mets:
    if met not in [x.metabolites for x in transport_w_measured_products]:
        if (met[:-2]+'_c') not in model.metabolites:
            print(met[:-2])
            l_not.append(met[:-2]+'_c')
        else:
            l.append(met[:-2]+'_c')
    else:
        good.append(met[:-2]+'_c')
# l_not # mets not in model
# l # mets in model but not transported
# good

mets not in model
C08276
thymd
mmal
mal__D
3gmp
mal__L


In [11]:
measured_met_export_rxns = list()
from cobra import Metabolite
            
for x in (l+l_not):
    rxn = universal.metabolites.get_by_id(x).reactions in transport_w_measured_products_UNI
    if rxn:
        print('no new reaction created - if this is printed, need to add boundaries')
        model.add_reactions([rxn])
    else:
        if x not in [y.id for y in model.metabolites]:
            if x in [y.id for y in universal.metabolites]:
                model.add_metabolites(universal.metabolites.get_by_id(x))
                model.metabolites.get_by_id(x).name = universal.metabolites.get_by_id(x).name
                model.metabolites.get_by_id(x).compartment = 'c'
                model.metabolites.get_by_id(x).formula = universal.metabolites.get_by_id(x).formula
            else:
                print(x+' not in universal')
        if x[:-2]+'_e' not in [y.id for y in model.metabolites]:
            if x[:-2]+'_e' in [y.id for y in universal.metabolites]:
                model.add_metabolites(universal.metabolites.get_by_id(x[:-2]+'_e'))
                model.metabolites.get_by_id(x[:-2]+'_e').name = universal.metabolites.get_by_id(x[:-2]+'_e').name
                model.metabolites.get_by_id(x[:-2]+'_e').compartment = 'e'
                model.metabolites.get_by_id(x[:-2]+'_e').formula = universal.metabolites.get_by_id(x[:-2]+'_e').formula
            else:
                print(x[:-2]+'_e'+' not in universal, so must be created')
                my_metabolite = Metabolite(x[:-2]+'_e')
                my_metabolite.name = universal.metabolites.get_by_id(x).name
                my_metabolite.compartment = 'e'
                my_metabolite.formula = universal.metabolites.get_by_id(x).formula
                model.add_metabolites(my_metabolite)
            model.add_boundary(model.metabolites.get_by_id(x[:-2]+'_e'), 
                               reaction_id= 'EX_' + x[:-2]+'_e',
                               type="ir_exchange", lb=0., ub=1000.0) ### CHECK
        rxn = cobra.Reaction()
        rxn.id = x[:-2]+'tex'
        rxn.name = x+' export'
        rxn.subsystem = 'transport'
        model.add_reactions([rxn])
        model.reactions.get_by_id(x[:-2]+'tex').build_reaction_from_string(x+'->'+x[:-2]+'_e')
        measured_met_export_rxns.append(rxn)
#         print(rxn.reaction)

# gap fill these for positive flux

6pgl_e not in universal, so must be created
g3p_e not in universal, so must be created
C08276_e not in universal, so must be created
mmal_e not in universal, so must be created


In [12]:
# cobra.io.write_sbml_model(pf_model, "test.xml")

cobra.io.save_json_model(model, "test.json")

In [13]:
##CHECK

for x in model.metabolites:
    if x.compartment == '':
        print(x)

In [14]:
# print(len(universal_model.reactions))
# for mets in universal_model.metabolites:
#     if mets.id.endswith('_e'):
#         if 'EX_'+mets.id not in [x.id for x in universal_model.reactions]:
#             universal_model.add_boundary(mets,'exchange')
# print(len(universal_model.reactions))

# l = list()
# for x in measured_met_export_rxns + transport_w_measured_products:
#     rx = x.copy()
#     if rx not in universal_model.reactions:
#         universal_model.add_reactions([rx])
#     universal_model.objective = universal_model.reactions.get_by_id(x.id)
#     if linear_reaction_coefficients(universal_model)[universal_model.reactions.get_by_id(x.id)] == 1:
#         if universal_model.slim_optimize() == 0:
#             print(x.id)
#             print('CANNOT BE SYNTHESIZED IN UNIVERSAL MODEL')
#             l.append(x.id)

In [15]:

from __future__ import absolute_import

from optlang.symbolics import add, Zero

from optlang.interface import OPTIMAL
from cobra.core import Model
from cobra.util import fix_objective_as_constraint

from warnings import warn

# UPDATED TO 0.13.0

class GapFiller(object):
    """Class for performing gap filling.
    This class implements gap filling based on a mixed-integer approach,
    very similar to that described in [1]_ and the 'no-growth but growth'
    part of [2]_ but with minor adjustments. In short, we add indicator
    variables for using the reactions in the universal model, z_i and then
    solve problem
    minimize \sum_i c_i * z_i
    s.t. Sv = 0
         v_o >= t
         lb_i <= v_i <= ub_i
         v_i = 0 if z_i = 0
    where lb, ub are the upper, lower flux bounds for reaction i, c_i is a
    cost parameter and the objective v_o is greater than the lower bound t.
    The default costs are 1 for reactions from the universal model, 100 for
    exchange (uptake) reactions added and 1 for added demand reactions.
    Note that this is a mixed-integer linear program and as such will
    expensive to solve for large models. Consider using alternatives [3]_
    such as CORDA instead [4,5]_.
    Parameters
    ----------
    model : cobra.Model
        The model to perform gap filling on.
    universal : cobra.Model
        A universal model with reactions that can be used to complete the
        model.
    lower_bound : float
        The minimally accepted flux for the objective in the filled model.
    penalties : dict, None
        A dictionary with keys being 'universal' (all reactions included in
        the universal model), 'exchange' and 'demand' (all additionally
        added exchange and demand reactions) for the three reaction types.
        Can also have reaction identifiers for reaction specific costs.
        Defaults are 1, 100 and 1 respectively.
    integer_threshold : float
        The threshold at which a value is considered non-zero (aka
        integrality threshold). If gapfilled models fail to validate,
        you may want to lower this value.
    exchange_reactions : bool
        Consider adding exchange (uptake) reactions for all metabolites
        in the model.
    demand_reactions : bool
        Consider adding demand reactions for all metabolites.
    References
    ----------
    .. [1] Reed, Jennifer L., Trina R. Patel, Keri H. Chen, Andrew R. Joyce,
       Margaret K. Applebee, Christopher D. Herring, Olivia T. Bui, Eric M.
       Knight, Stephen S. Fong, and Bernhard O. Palsson. “Systems Approach
       to Refining Genome Annotation.” Proceedings of the National Academy
       of Sciences 103, no. 46 (2006): 17480–17484.
       [2] Kumar, Vinay Satish, and Costas D. Maranas. “GrowMatch: An
       Automated Method for Reconciling In Silico/In Vivo Growth
       Predictions.” Edited by Christos A. Ouzounis. PLoS Computational
       Biology 5, no. 3 (March 13, 2009): e1000308.
       doi:10.1371/journal.pcbi.1000308.
       [3] http://opencobra.github.io/cobrapy/tags/gapfilling/
       [4] Schultz, André, and Amina A. Qutub. “Reconstruction of
       Tissue-Specific Metabolic Networks Using CORDA.” Edited by Costas D.
       Maranas. PLOS Computational Biology 12, no. 3 (March 4, 2016):
       e1004808. doi:10.1371/journal.pcbi.1004808.
       [5] Diener, Christian https://github.com/cdiener/corda
     """

    def __init__(self, model, universal=None, lower_bound=0.05,
                 penalties=None, exchange_reactions=False,
                 demand_reactions=True, integer_threshold=1e-6):
        self.original_model = model
        self.lower_bound = lower_bound
        self.model = model.copy()
        tolerances = self.model.solver.configuration.tolerances
        tolerances.integrality = integer_threshold
        self.universal = universal.copy() if universal else Model('universal')
        self.penalties = dict(universal=1, exchange=100, demand=1)
        if penalties is not None:
            self.penalties.update(penalties)
        self.integer_threshold = integer_threshold
        self.indicators = list()
        self.costs = dict()
        self.extend_model(exchange_reactions, demand_reactions)
        fix_objective_as_constraint(self.model, bound=lower_bound)
        self.add_switches_and_objective()

    def extend_model(self, exchange_reactions=False, demand_reactions=True):
        """Extend gapfilling model.
        Add reactions from universal model and optionally exchange and
        demand reactions for all metabolites in the model to perform
        gapfilling on.
        Parameters
        ----------
        exchange_reactions : bool
            Consider adding exchange (uptake) reactions for all metabolites
            in the model.
        demand_reactions : bool
            Consider adding demand reactions for all metabolites.
        """
        for rxn in self.universal.reactions:
            rxn.gapfilling_type = 'universal'
        new_metabolites = self.universal.metabolites.query( #### NEW ####
                lambda metabolite: metabolite not in self.model.metabolites
        )
        self.model.add_metabolites(new_metabolites) #### NEW ####
        for met in self.model.metabolites:    
            if exchange_reactions:
                rxn = self.universal.add_boundary(
                    met, type='exchange_smiley', lb=-1000, ub=0,
                    reaction_id='EX_{}'.format(met.id))
                rxn.gapfilling_type = 'exchange'
            if demand_reactions:
                rxn = self.universal.add_boundary(
                    met, type='demand_smiley', lb=0, ub=1000,
                    reaction_id='DM_{}'.format(met.id))
                rxn.gapfilling_type = 'demand'

        new_reactions = self.universal.reactions.query(
            lambda reaction: reaction not in self.model.reactions
        )
        self.model.add_reactions(new_reactions)

    def update_costs(self):
        """Update the coefficients for the indicator variables in the objective.
        Done incrementally so that second time the function is called,
        active indicators in the current solutions gets higher cost than the
        unused indicators.
        """
        for var in self.indicators:
            if var not in self.costs:
                self.costs[var] = var.cost
            else:
                if var._get_primal() > self.integer_threshold:
                    self.costs[var] += var.cost
        self.model.objective.set_linear_coefficients(self.costs)

    def add_switches_and_objective(self):
        """ Update gapfilling model with switches and the indicator objective.
        """
        constraints = list()
        big_m = max(max(abs(b) for b in r.bounds)
                    for r in self.model.reactions)
        prob = self.model.problem
        for rxn in self.model.reactions:
            if not hasattr(rxn, 'gapfilling_type'):# or rxn.id.startswith('DM_'):
                continue
            indicator = prob.Variable(
                name='indicator_{}'.format(rxn.id), lb=0, ub=1, type='binary')
            if rxn.id in self.penalties:
                indicator.cost = self.penalties[rxn.id]
            else:
                indicator.cost = self.penalties[rxn.gapfilling_type]
            indicator.rxn_id = rxn.id
            self.indicators.append(indicator)

            # if z = 1 v_i is allowed non-zero
            # v_i - Mz <= 0   and   v_i + Mz >= 0
            constraint_lb = prob.Constraint(
                rxn.flux_expression - big_m * indicator, ub=0,
                name='constraint_lb_{}'.format(rxn.id), sloppy=True)
            constraint_ub = prob.Constraint(
                rxn.flux_expression + big_m * indicator, lb=0,
                name='constraint_ub_{}'.format(rxn.id), sloppy=True)

            constraints.extend([constraint_lb, constraint_ub])

        self.model.add_cons_vars(self.indicators)
        self.model.add_cons_vars(constraints, sloppy=True)
        self.model.objective = prob.Objective(
            #Add(*self.indicators), direction='min') # UPDATED TO 0.13.0
            Zero, direction='min', sloppy=True)
        self.model.objective.set_linear_coefficients({ # UPDATED TO 0.13.0
            i: 1 for i in self.indicators})
        self.update_costs()

    def fill(self, iterations=1):
        """Perform the gapfilling by iteratively solving the model, updating
        the costs and recording the used reactions.
        Parameters
        ----------
        iterations : int
            The number of rounds of gapfilling to perform. For every
            iteration, the penalty for every used reaction increases
            linearly. This way, the algorithm is encouraged to search for
            alternative solutions which may include previously used
            reactions. I.e., with enough iterations pathways including 10
            steps will eventually be reported even if the shortest pathway
            is a single reaction.
        Returns
        -------
        iterable
            A list of lists where each element is a list reactions that were
            used to gapfill the model.
        Raises
        ------
        RuntimeError
            If the model fails to be validated (i.e. the original model with
            the proposed reactions added, still cannot get the required flux
            through the objective).
        """
        used_reactions = list()
        for i in range(iterations):
            self.model.slim_optimize(error_value=None,
                                     message='gapfilling optimization failed')
            solution = [self.model.reactions.get_by_id(ind.rxn_id)
                        for ind in self.indicators if
                        ind._get_primal() > self.integer_threshold]
            if not self.validate(solution):
                raise RuntimeError('failed to validate gapfilled model, '
                                   'try lowering the integer_threshold')
            used_reactions.append(solution)
            self.update_costs()
        return used_reactions

    def validate(self, reactions):
        with self.original_model as model:
            model.add_reactions(reactions)
            mets  = [x.metabolites for x in reactions] #### ADDED NEW ####
            all_keys = set().union(*(d.keys() for d in mets)) #### ADDED NEW ####
            model.add_metabolites(all_keys) #### ADDED NEW ####
            model.slim_optimize()
            return (model.solver.status == OPTIMAL and
                    model.solver.objective.value >= self.lower_bound)


def gapfill2(model, universal=None, lower_bound=0.05,
            penalties=None, demand_reactions=True, exchange_reactions=False,
            iterations=1):
    """Perform gapfilling on a model.
    See documentation for the class GapFiller.
    Parameters
    ----------
    model : cobra.Model
        The model to perform gap filling on.
    universal : cobra.Model, None
        A universal model with reactions that can be used to complete the
        model. Only gapfill considering demand and exchange reactions if
        left missing.
    lower_bound : float
        The minimally accepted flux for the objective in the filled model.
    penalties : dict, None
        A dictionary with keys being 'universal' (all reactions included in
        the universal model), 'exchange' and 'demand' (all additionally
        added exchange and demand reactions) for the three reaction types.
        Can also have reaction identifiers for reaction specific costs.
        Defaults are 1, 100 and 1 respectively.
    iterations : int
        The number of rounds of gapfilling to perform. For every iteration,
        the penalty for every used reaction increases linearly. This way,
        the algorithm is encouraged to search for alternative solutions
        which may include previously used reactions. I.e., with enough
        iterations pathways including 10 steps will eventually be reported
        even if the shortest pathway is a single reaction.
    exchange_reactions : bool
        Consider adding exchange (uptake) reactions for all metabolites
        in the model.
    demand_reactions : bool
        Consider adding demand reactions for all metabolites.
    Returns
    -------
    iterable
        list of lists with on set of reactions that completes the model per
        requested iteration.
    Examples
    --------
    >>> import cobra.test as ct
    >>> from cobra import Model
    >>> from cobra.flux_analysis import gapfill
    >>> model = ct.create_test_model("salmonella")
    >>> universal = Model('universal')
    >>> universal.add_reactions(model.reactions.GF6PTA.copy())
    >>> model.remove_reactions([model.reactions.GF6PTA])
    >>> gapfill(model, universal)
    """
    gapfiller = GapFiller(model, universal=universal,
                          lower_bound=lower_bound, penalties=penalties,
                          demand_reactions=demand_reactions,
                          exchange_reactions=exchange_reactions)
    return gapfiller.fill(iterations=iterations)

model.solver


In [16]:
met = Metabolite('C08276_e',formula='', name=universal.metabolites.get_by_id('C08276_c').name,compartment='e')
universal.add_metabolites(met)
universal.add_boundary(met, type="exchange")
met = Metabolite('6pgl_e',formula='', name=universal.metabolites.get_by_id('6pgl_c').name,compartment='e')
universal.add_metabolites(met)
universal.add_boundary(met, type="exchange")
met = Metabolite('g3p_e',formula='', name=universal.metabolites.get_by_id('g3p_c').name,compartment='e')
universal.add_metabolites(met)
universal.add_boundary(met, type="exchange")
met = Metabolite('mmal_e',formula='', name=universal.metabolites.get_by_id('mmal_c').name,compartment='e')
universal.add_metabolites(met)
universal.add_boundary(met, type="exchange")


Reaction identifier,EX_mmal_e
Name,Methylmalonate C4H4O4 exchange
Memory address,0x01234a32b0
Stoichiometry,mmal_e <=> Methylmalonate C4H4O4 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [17]:

# # gapfill based on excretion of these metabolites
# result = dict()
# with model:
#     model.solver.configuration.timeout = 20
#     for x in measured_met_export_rxns + transport_w_measured_products:
#         if x.id not in result.keys():
#             print(x.id)
#             model.objective = model.reactions.get_by_id(x.id)
#             if cobra.util.solver.linear_reaction_coefficients(model)[x] == 1:
#                 print(model.optimize().f)
#                 if model.optimize().f < 0.1:
#                     result[x.id] = gapfill2(model, universal= universal, demand_reactions = False, 
#                                             exchange_reactions = False, iterations=10)
#                 else:
#                     result[x.id] = 'not needed'
#             else:
#                 print('error in setting objective')
#             print(result[x.id])
        
# pd.DataFrame.from_dict(data=result, orient='index').to_csv('metabolomics_gapfilling_results_july23.csv', header=False)

# keys_to_remove = [key for key, value in result.items() if value == 'not needed']
# for key in keys_to_remove:
#     del result[key]

In [18]:
# model_0 = model.copy()

# l = list()
# df = pd.DataFrame.from_dict(data=result, orient='index')
# for x in range(0,10,1): 
#     for y in df[x].index:
#         for w in range(0,len(df[x][y]),1): # df[itteration number][gapfill query name]
#             l.append(df[x][y][w].id)
# rxn = set(l)

# counter = dict.fromkeys(rxn, 0)
# for r in rxn:
#     for x in range(0,10,1):
#         for y in df[x].index:
#             for w in range(0,len(df[x][y]),1): # df[itteration number][gapfill query name]
#                 if df[x][y][w].id == r:
#                     counter[r] = counter[r] + 1
        
# counter # this shows how many solutions a reaction was used in
# counter_df = pd.DataFrame.from_dict(data=counter, orient='index')
# counter_df.to_csv('metabolomics_gapfilling_results_quantitative_july23.csv', header=False)

# for key, value in result.items():
#     for y in range(0,10,1):
#         for x in range(0,len(value[y]),1):
#             reactio = universal.reactions.get_by_id(value[y][x].id)
#             if reactio.id not in [r.id for r in model_0.reactions]:
#                 model_0.add_reaction(reactio)
#                 model_0.reactions.get_by_id(value[y][x].id).notes = {'metabolomics gapfilling confidence score, 10 iterations': counter_df.at[value[y][x].id, 0]}

In [19]:
# # save all scores as json
# result_text = dict()
# for key,value in result.items():
#     print(key)
#     print(value)
#     li2 = list()
#     for i in range(0,len(value),1):
#         li1 = list()
#         for j in range(0,len(value[i]),1):
#             li1.append(value[i][j].id)
#         li2.append(li1)
#     result_text[key] = li2
# result_text
# import json
# class JSONEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if hasattr(obj, 'to_json'):
#             return obj.to_json(orient='records')
#         return json.JSONEncoder.default(self, obj)
# with open('/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/data/metabolomics_gapfilling_results_july25.json', 'w') as fp:
#     json.dump(result_text, fp, cls=JSONEncoder)
    
# result = json.load(open('/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/data/metabolomics_gapfilling_results_july25.json'))

In [20]:
import json
result = json.load(open('/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/data/metabolomics_gapfilling_results_july25.json'))


In [21]:
l = list()
for key, value in result.items():
    for i in range(0,len(value),1):
        for j in range(0,len(value[i]),1):
            l.append(value[i][j])
met_list = list()
for x in l:
    rxn = universal.reactions.get_by_id(x).copy()
    for met in rxn.metabolites:
        if met.id not in [m.id for m in model.metabolites]:
            met_list.append(met)
#     if rxn.id not in [r.id for r in model.reactions]:
#         model.add_reactions([rxn])
os.chdir(data_path+'/other_models')
# cobra.io.save_json_model(model, "iPfal18.json")

In [31]:
[universal.reactions.get_by_id(r).name for r in set(l)]

['FRDO2r',
 '3-Hydroxy-2-Methylpropanoyl Coenzyme A Hydrolase',
 'Thymidine transport (1:2 Na/Thymd cotransport)',
 'Fumarase  mitochondrial',
 'Ubiquinol-6 cytochrome c reductase, Complex III',
 'CO transporter via diffusion',
 '(S)-3-Hydroxyisobutyryl Coenzyme A Hydro-Lyase Valine, Leucine And Isoleucine Degradation',
 'Proton transport via diffusion (extracellular to periplasm)',
 '3 hydroxyacyl CoA dehydrogenase',
 'Carbon monoxide dehydrogenase / acetyl-CoA synthase 2',
 'Malate transport via proton symport (3 H)',
 "5'-nucleotidase (dTMP)",
 'EX co LPAREN e RPAREN ',
 '3-hydroxyisobutyryl-CoA hydrolase, mitochondrial',
 '5-Methylthio-5-deoxy-D-ribulose 1-phosphate dehydratase',
 'Sodium proton antiporter (H:NA is 2)',
 '2-oxoisovalerate dehydrogenase (acylating; 3-methyl-2-oxobutanoate)',
 'Fumarate reductase  cytosolicmitochondrial',
 'Nitrate reductase (ferredoxin)',
 'D-Malate transport via proton symport (2 H) (periplasm)',
 'Deoxyadenosine kinase',
 'Aldehyde dehydrogenase  

In [36]:
len(set(l))
len([r for r in set(l) if r not in [x.id for x in model.reactions]])

68

In [23]:
set([m.name for m in met_list])

{' S  Methylmalonate semialdehyde C4H5O3',
 '(S)-3-Hydroxyisobutyryl-CoA',
 '2,3-diketo-5-methylthio-1-phosphopentane',
 '2-Methylprop-2-enoyl-CoA',
 '3-Hydroxy-2-methylpropanoate',
 '3-Hydroxyisobutyryl Coenzyme A',
 '5-Methylthio-5-deoxy-D-ribose 1-phosphate',
 '5-Methylthio-5-deoxy-D-ribulose 1-phosphate',
 '5-Methylthio-D-ribose',
 'Carbon monoxide',
 'D-Malate',
 'Ferredoxin (oxidized form 4:2)',
 'Ferredoxin (oxidized) 2[4Fe-4S]',
 'Ferredoxin (reduced form 4:2)',
 'Ferredoxin (reduced) 2[4Fe-4S]',
 'Ferricytochrome c',
 'Ferrocytochrome C',
 'Flavin adenine dinucleotide oxidized',
 'Flavin adenine dinucleotide reduced',
 'Glyoxylate',
 'H+',
 'Hydrogen',
 'Isobutyryl-CoA',
 'L-Lactate',
 'L-Malate',
 'Oxidized adrenal ferredoxin',
 'Reduced adrenal ferredoxin',
 'Sodium',
 'Ubiquinol-10',
 'Ubiquinone-10'}

In [24]:
[x.reaction for x in model.reactions if x.id not in [r.id for r in og_pf.reactions]]

['5mta_e --> 5mta_c',
 '5mta_e --> ',
 'gtp_e --> ',
 'gtp_e --> gtp_c',
 'dttp_e --> ',
 'dttp_e --> dttp_c',
 'gdp_e --> ',
 'gdp_c --> gdp_e',
 'damp_e --> ',
 'damp_c --> damp_e',
 '6pgl_e --> ',
 '6pgl_c --> 6pgl_e',
 'pep_e --> ',
 'pep_c --> pep_e',
 'udp_e --> ',
 'udp_c --> udp_e',
 'dtmp_e --> ',
 'dtmp_c --> dtmp_e',
 '3pg_e --> ',
 '3pg_c --> 3pg_e',
 'nac_c --> nac_e',
 'cbasp_e --> ',
 'cbasp_c --> cbasp_e',
 'fum_c --> fum_e',
 'succ_c --> succ_e',
 'adp_e --> ',
 'adp_c --> adp_e',
 'amp_e --> ',
 'amp_c --> amp_e',
 'g3p_e --> ',
 'g3p_c --> g3p_e',
 'udpg_e --> ',
 'udpg_c --> udpg_e',
 'imp_e --> ',
 'imp_c --> imp_e',
 'dtdp_e --> ',
 'dtdp_c --> dtdp_e',
 'cdp_e --> ',
 'cdp_c --> cdp_e',
 'glyc3p_e --> ',
 'glyc3p_c --> glyc3p_e',
 'ump_e --> ',
 'ump_c --> ump_e',
 'xtsn_c --> xtsn_e',
 'dgmp_e --> ',
 'dgmp_c --> dgmp_e',
 'akg_c --> akg_e',
 'C08276_e --> ',
 'C08276_c --> C08276_e',
 'thymd_e --> ',
 'thymd_c --> thymd_e',
 'mmal_e --> ',
 'mmal_c --> mmal_e',

In [25]:
set([x.name for x in model.metabolites if x.id[:-2] not in [r.id[:-2] for r in og_pf.metabolites]])

{'3-(methylthio)propionate',
 'D-Malate',
 'Guanosine 3  phosphate C10H12N5O8P',
 'L-Malate',
 'Methylmalonate C4H4O4',
 'Thymidine C10H14N2O5'}

In [26]:
#Object of type 'int64' is not JSON serializable 

# there are int64 objects that should be converted to int

In [27]:
print('HIGH CONFIDENCE RXNS')
for x in counter_df[counter_df[0] ==10].index:
    print(x)
    print(model_0.reactions.get_by_id(x).reaction)

# confirm confidence scores were added
model_0.reactions.get_by_id('r0779').notes

HIGH CONFIDENCE RXNS


NameError: name 'counter_df' is not defined

In [ ]:
og_model = og_pf
for x in [og_model, model,model_0]:
    print('rxns:')
    print(len(x.reactions))
    print('mets:')
    print(len(x.metabolites))
    print(x.slim_optimize())
    print('')
    

In [ ]:
(1323-1196)


In [ ]:
transport_reactions = list()
# see how many transport reactions were added
for x in counter_df.index:
    rxn = model_0.reactions.get_by_id(x)
    # Collecting criteria to classify transporters by.
    rxn_reactants = set([met.formula for met in rxn.reactants])
    rxn_products = set([met.formula for met in rxn.products])
    # Looking for formulas that stay the same on both side of the reaction.
    transported_mets = [formula for formula in rxn_reactants if formula in rxn_products]
    # Collect information on the elemental differences between
    # compartments in the reaction.
    delta_dicts = find_transported_elements(rxn)
    non_zero_array = [v for (k, v) in iteritems(delta_dicts) if v != 0]
    # Weeding out reactions such as oxidoreductases where no net
    # transport of Hydrogen is occurring, but rather just an exchange of
    # electrons or charges effecting a change in protonation.
    if set(transported_mets) != set('H') and list(
        delta_dicts.keys()
    ) == ['H']:
        pass
    # All other reactions for which the amount of transported elements is
    # not zero, which are not part of the model's exchange nor
    # biomass reactions, are defined as transport reactions.
    # This includes reactions where the transported metabolite reacts with
    # a carrier molecule.
    
    ## updated 
    elif sum(non_zero_array) and rxn not in find_biomass_reaction(model):
        if rxn.id.startswith('EX_'):
            temp = rxn.id
        else:
            transport_reactions.append(rxn)
(3+len(transport_reactions))/(len(model_0.reactions)-len(og_model.reactions))

In [ ]:
# cobra.io.write_sbml_model(model_0,"pf_curated_and_gapfilled.xml")
# cobra.io.save_json_model(model_0, "pf_curated_and_gapfilled.json")

In [ ]:
model_0